In [1]:
import pandas as pd
import numpy as np
import math

#Map interactive
import folium as f
from folium.plugins import MarkerCluster

#Map pour connection
import matplotlib.pyplot as plt
%matplotlib inline

#Calcul great circle
import pyproj

# To Load File
import bz2
import pickle

from IPython.display import IFrame
import mercury as mr # for widgets


In [2]:
# TEST APP
show_code = mr.Checkbox(label="Voir le code", value=False)
app = mr.App(title="Analyse géographique", 
            description="Découvrez les liaisons aériennes entre les aéroports du monde entier", 
            show_code=show_code.value,
            continuous_update=False,
            allow_download=False)

mercury.Checkbox

mercury.App

In [3]:
# Importation des données

# Les vols
with bz2.BZ2File('./All_DfVols.bz2', 'r') as file:
    df_vols, _, _ = pickle.load(file)

# Load all dictionnaries from file using pickle using bz2 compression
with bz2.BZ2File('AllData_In_Dict.bz2', 'r') as file:
    dict_compagnies, dict_aeroports, dict_pays = pickle.load(file)

In [4]:
# Fonction de distance
def coordDepuisAeroport(iata : str):
    '''
    Fonction : Recherche les coordonnées GPS d'un aéroport
    Retour : lat : float ,lon : float --> Latitude & longitude de l'aéroport
    '''
    lat = float(dict_aeroports[iata]['Lat'])
    lon = float(dict_aeroports[iata]['Lon'])
    return lat,lon

def DistGrandCercle(lat1 : float, lon1 : float, lat2 : float, lon2 : float):
    '''
    Fonction :  Calcul la longueur du grand cercle (= cercle tracé à la surface 
                d'une sphère qui a le même diamètre qu'elle) entre 2 coordonnées
    Retour : Distance en Km
    '''
    #Conversion en radian
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    #Delta des coordonnées
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    #Rayon terrestre (en km)
    r = 6367.0

    #Formule d'Haversine
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    return 2 * r * math.asin(math.sqrt(a))


def DistGrandCercleICAO(icao1 : str, icao2 : str):
    '''
    Fonction :  Calcul la longueur du grand cercle (= cercle tracé à la surface 
                d'une sphère qui a le même diamètre qu'elle) entre 2 aéroports
    Retour : Distance en Km
    '''
    #Récupération des coordonnées des aéroports
    latDep, lonDep = coordDepuisAeroport(icao1)
    latArr, lonArr = coordDepuisAeroport(icao2)

    #On calcule la distance les séparant
    return DistGrandCercle(latDep,lonDep,latArr,lonArr)

def Dist(x1,y1,x2,y2):
    '''
    Fonction : Calcul la distance euclidienne entre deux points
    Retour : Distance
    '''
    return math.sqrt((y2-y1)**2+(x2-x1)**2)

# Fonction de calcul de coordonnées
def vectGrandCercle(latDep : float, lonDep : float, latArr : float, lonArr : float):
    '''
    Fonction : Calcul un ensemble de coordonnées permettant de tracer un grand cercle terrestre
    Return : 2 listes de coordonnées de forme [[lat,lon],[lat,lon]]
    '''

    #On initialise le module de calcul
    g = pyproj.Geod(ellps='WGS84')
    (az12, az21, dist) = g.inv(lonDep, latDep, lonArr, latArr)

    # On découpe le chemin entre 2 coordonnée avec des segments <= 100 km
    lonlats = g.npts(lonDep, latDep, lonArr, latArr,
                    1 + int(dist / 100000))

    #Mise en forme
    v1 = []
    v1.append([latDep,lonDep])
    v2 = []
    _lon = lonDep
    _lat = latDep
    horscadre = False

    # On parcourt les segments afin de créer un vecteur de coordonnées [lat,lon]
    # et non [lon,lat] comme retourne la fonction g.npts
    # La notion "horscadre" défini le dépassement d'un planisphère (à l'est ou à l'ouest)
    # La longitude passe de -180 à l'ouest à + 180 à l'est
    # On détecte ce dépassement avec un calcul de distance euclidienne Dist(_lat,_lon,lat,lon) < 4
    for lon, lat in lonlats:
        if(not(horscadre) and Dist(_lat,_lon,lat,lon) < 4):        
            v1.append([lat,lon])
        else:
            horscadre = True
            v2.append([lat,lon])
        _lon = lon
        _lat = lat
    
    if(horscadre):
        v2.append([latArr,lonArr])
    else:
        v1.append([latArr,lonArr])

    return v1,v2

In [5]:
# Filtre des dictionnaires
def filtreDictAeroports(colonne : str, valeur : str):
    '''
    Fonction : Filtre les clés du dictionnaire en fonction d'une valeur 
    présente dans les valeurs associées.
    Return : Liste de code ICAO des aéroports
    '''
    Listtmp = []
    for (key, value) in dict_aeroports.items():
        if value[colonne] == valeur:
            Listtmp.append(key)
        if valeur == '':
            Listtmp.append(value[colonne])
    return Listtmp

def filtreDictCompagnies(colonne : str, valeur : str):
    '''
    Fonction : Filtre les clés du dictionnaire en fonction d'une valeur 
    présente dans les valeurs associées.
    Return : Liste de code ICAO des compagnies
    '''
    Listtmp = []
    for (key, value) in dict_compagnies .items():
        if value[colonne] == valeur:
            Listtmp.append(key)
        if valeur == '':
            Listtmp.append(value[colonne])

    return Listtmp

In [6]:
dict_route = {}

g = df_vols.groupby(['Depart','Arrivee'])
taille_groupe = g.size().items()

for liaison, nbrVols in taille_groupe:
    #Si la liaison est déjà dans le dict, on incrémente le nbr de vols
    if(frozenset(liaison) in dict_route):
        dict_route[frozenset(liaison)]['Vols'] += nbrVols
    #Sinon, on crée une nouvelle liaison
    else:
        dict_route[frozenset(liaison)] = {'Dist' : DistGrandCercleICAO(liaison[0],liaison[1]), 'Vols' : nbrVols}

### Emplacement des aéroports mondiaux

In [7]:
# m = f.Map(
#     location = [0, 0],
#     zoom_start = 2
# )

# cluster = MarkerCluster(name='Aeroport').add_to(m)
# f.LayerControl().add_to(m)


# for liste in dict_aeroports.values():
#     f.Marker(
#         location = [liste['Lat'], liste['Lon']], #Position sur la carte
#         popup= "<b>Nom : </b>" + liste['Name'] + """<br />
#         <b>Code :</b> """ + liste['Iata'] + """<br />
#         <b>Coordonnées :</b> (""" + str(liste['Lat']) + ";" + str(liste['Lon']) + """)<br />
#         <b>City : </b>""" + str(liste['City']) + """<br />
#         <b>Pays : </b>""" + liste['Country'],
#         tooltip=liste['City'], #hoover over 
#         icon= f.Icon(icon="plane")
#     ).add_to(cluster)
# m.save('map.html')
# m

In [8]:
# La carte a été pré-enregistré pour éviter de surcharger le notebook.
IFrame(src='map.html', width=900, height=600)

In [9]:
list_Name_compagnies = filtreDictCompagnies('Name', '')
list_Name_compagnies.append(' ')
list_Name_compagnies.sort()

In [10]:
S_nameAirport = mr.Select(label="Selectionner un aéroport *", value="Lyon Saint Exupery Airport", choices=filtreDictAeroports('Name', ''))
S_nameCompagnie = mr.Select(label="Selectionner une compagnie",  value=' ', choices=list_Name_compagnies)

mercury.Select

mercury.Select

In [11]:
if S_nameCompagnie.value != ' ':
    mr.Md(f"### Liaisons depuis _{S_nameAirport.value}_ avec la compagnie _{S_nameCompagnie.value}_")
else:
    mr.Md(f"### Liaisons depuis _{S_nameAirport.value}_")

### Liaisons depuis _Lyon Saint Exupery Airport_

In [12]:
def checkWidget(nameAirport, nameCompagnie):
    #On récupère le Code_ICAO associé aux paramètres
    List_pays = filtreDictAeroports('Name',nameAirport.value)
    List_compagnie = filtreDictCompagnies('Name',nameCompagnie.value)

    #On convertit le format list en format str
    if(len(List_pays) != 1):
        print("Veuiller choisir un aéroport")
        return
    else:
        pays = List_pays[0]
        if(len(List_compagnie) == 1):
            compagnie = List_compagnie[0]
        else:
            compagnie = ""
    return pays, compagnie
    
def map_connection(aeroport,compagnie,m):
    '''
    Fonction : Trace une carte de connexion entre un aéroport et toutes ses distinations
    '''
    Line_monde = f.FeatureGroup(name = "International")
    Line_continent = f.FeatureGroup(name = "Continental")
    Line_pays = f.FeatureGroup(name = "National") 

    aeroport, compagnie = checkWidget(aeroport, compagnie)  

    #On applique un filtre sur les données en fonction des paramètres
    if(aeroport == ""):
        df_tmp = df_vols[(df_vols['Code_ICAO'] == compagnie)]
    elif(compagnie == ""):
        df_tmp = df_vols[(df_vols['Depart'] == aeroport)]
    else:
        df_tmp = df_vols[(df_vols['Depart'] == aeroport) & (df_vols['Code_ICAO'] == compagnie)]

    #On trace la liaison pour toutes les vols répondant aux filtres
    for _, ligne in df_tmp.iterrows():
        # Certains aéroports n'ont pas de correspondance dans le dict. (API ne renvoyant pas les bonnes valeurs) 
        # Pour éviter l'erreur -> try puis except : pass
        try:
            latDep, lonDep = coordDepuisAeroport(ligne.Depart)
            latArr, lonArr = coordDepuisAeroport(ligne.Arrivee)

            #Calcul des chemins à afficher
            v2 = []
            v1,v2 = vectGrandCercle(latDep,lonDep,latArr,lonArr)

            #Test vol national ou international
            if(dict_aeroports[ligne.Depart]['Country'] == dict_aeroports[ligne.Arrivee]['Country']):
                Line_group = Line_pays
                col_ligne = "#80F0A3"
                col_marker = "#008329"
            elif(dict_pays[dict_aeroports[ligne.Depart]['Country']] == dict_pays[dict_aeroports[ligne.Arrivee]['Country']] ):
                Line_group = Line_continent
                col_ligne = "#F6FF28"
                col_marker = "#FE7700"
            else:
                Line_group = Line_monde
                col_ligne = "#563DFF"
                col_marker = "#0004A8"


            #-------------------
            # Traçage des lignes
            f.PolyLine(locations=v1,weight=1, color=col_ligne).add_to(Line_group)
            if(len(v2) != 0):
                f.PolyLine(locations=v2,weight=1, color=col_ligne).add_to(Line_group)
            
            f.CircleMarker(location = [latArr, lonArr],
                            radius = 1, 
                            color = col_marker, 
                            tooltip=dict_aeroports[ligne.Arrivee]['City'],
                            popup="<b>Nom : </b>" + dict_aeroports[ligne.Arrivee]['Name'] + """<br />
                                    <b>Code :</b> """ + dict_aeroports[ligne.Arrivee]['Iata'] + """<br />
                                    <b>Coordonnées :</b> (""" + str(dict_aeroports[ligne.Arrivee]['Lat']) + ";" + str(dict_aeroports[ligne.Arrivee]['Lon']) + """)<br />
                                    <b>City : </b>""" + str(dict_aeroports[ligne.Arrivee]['City']) + """<br />
                                    <b>Pays : </b>""" + dict_aeroports[ligne.Arrivee]['Country']).add_to(Line_group)
        except:
            pass

    #Ajout des layers à la carte
    Line_monde.add_to(m)
    Line_continent.add_to(m)
    Line_pays.add_to(m)

#Paramètre de la carte
m = f.Map(
    location = [0, 0],
    zoom_start = 2.3,
    min_zoom  = 2,
    min_lot=-180,
    max_lot=180,
    min_lat=-90,
    max_lat=90,
    max_bounds=True,
)
f.TileLayer('cartodbdark_matter').add_to(m)
#Rentrer un CODE aéroport et optionnellement un CODE compagnie 
map_connection(S_nameAirport,S_nameCompagnie,m)
f.LayerControl().add_to(m)

m

### Liaisons nationales

In [13]:
S_namePays = mr.Select(label="Selectionner un pays *", value="Germany", choices=list(dict_pays.keys()))

mercury.Select

In [14]:
mr.Md(f"#### Pour {S_namePays.value}")

#### Pour Germany

In [15]:
def map_connectionPays(pays,m):
    '''
    Fonction : Dessine les liaisons entre différentes aéroports au sein d'un même pays
    '''
    
    #Code ICAO des aéroports dans le pays en question
    listAeroportPays = filtreDictAeroports('Country',pays)

    #Filtrage des vols en fonction des aéroports possibles
    df_volsFiltre = df_vols[df_vols["Depart"].isin(listAeroportPays) & df_vols["Arrivee"].isin(listAeroportPays)]

    #Nombre de vols par aéroport (Depart + Arrivee)
    Serie_nbrVolsParAeroport =  pd.concat([df_volsFiltre['Depart'],df_volsFiltre['Arrivee']]).value_counts()
    #On groupe par liaison puis on on ajoute la colonne 'count' pour savoir combien de vols existent par liaison
    df_GroupLiaison = df_volsFiltre.groupby(['Depart','Arrivee']).size().reset_index(name="Count")
    
    #Nombre de départ par aéroport (Un départ = une destination car plus de doublons dans le couple [Depart,Arrivee])
    Serie_nbrDestinationParAeroport =  df_GroupLiaison['Depart'].value_counts()

    
    #Création des Layers    
    tabLine = []
    maxLigne = df_GroupLiaison['Count'].max()
    _max = maxLigne/5
    #-0.1 pour ne pas créer un nouveau bloc en cas de division entière
    for i in range(0,int(_max-0.1) + 1):
        tabLine.append(f.FeatureGroup(name = "Entre " + str(int(i*5) + 1) + " et " +
                                                str((int((i+1)*5))) + " liaisons"))

    for _, ligne in df_GroupLiaison.iterrows():
        #Certains aéroports n'ont pas de correspondance dans le dict. Pour éviter l'erreur -> try puis pass
        try:
            latDep, lonDep = coordDepuisAeroport(ligne.Depart)
            latArr, lonArr = coordDepuisAeroport(ligne.Arrivee)

            #Calcul des chemins à afficher
            v2 = []
            v1,v2 = vectGrandCercle(latDep,lonDep,latArr,lonArr)
            
            
            #-------------------
            f.PolyLine(locations=v1,weight=0.2 +ligne.Count/5, color='blue', opacity=0.5,
                        popup="<b>" + dict_aeroports[ligne.Depart]['City'] + " --> " +
                                dict_aeroports[ligne.Arrivee]['City'] +  "</b><br/>" +
                                "<b>Liaisons : </b>" + str(ligne.Count)
                        ).add_to(tabLine[int(ligne.Count/5 - 0.1)])
            
            if(len(v2) != 0):
                f.PolyLine(locations=v2,weight=0.2 +ligne.Count/5, color='blue', opacity=0.5,
                        popup="<b>" + dict_aeroports[ligne.Depart]['City'] + " --> " +
                                dict_aeroports[ligne.Arrivee]['City'] +  "</b><br/>" +
                                "<b>Liaisons : </b>" + str(ligne.Count)
                        ).add_to(tabLine[int(ligne.Count/5 - 0.1)])

        except:
            print("Erreur entre ", ligne.Depart, " et ", ligne.Arrivee, " -- ", ligne.Count)
            pass
    
    #Ajout des Layers:
    for i in range(0,int(_max-0.1) + 1):
        tabLine[i].add_to(m)

    for ligne in Serie_nbrVolsParAeroport.items():

        if ligne[0] in Serie_nbrDestinationParAeroport.index:
            nbrDest = Serie_nbrDestinationParAeroport[ligne[0]]
        else:
            nbrDest = 0  
        
        Text = f.IFrame("<b>" + dict_aeroports[ligne[0]]['Name'] + """</b><br />
                        <b>City : </b>""" + str(dict_aeroports[ligne[0]]['City']) + """<br />
                        <b>Pays : </b>""" + dict_aeroports[ligne[0]]['Country'] + """<br />
                        <b>Code :</b> """ + dict_aeroports[ligne[0]]['Iata'] + """<br/><br/>
                        <b>Destinations : </b> """ + str(nbrDest))

        #Circle vs CircleMarker
        f.CircleMarker(location= [dict_aeroports[ligne[0]]['Lat'],dict_aeroports[ligne[0]]['Lon']], 
                        radius = 1 + ligne[1]/35, 
                        color='red', 
                        fill_color='red',
                        fill_opacity=1,
                        tooltip=dict_aeroports[ligne[0]]['Name'],                        
                        popup = f.Popup(Text, min_width = 250, max_width = 400)
                        ).add_to(m)  


#Décommenter pour afficher la carte (Voir cellule suivante pour les widgets)

#Paramètre de la carte

pays = S_namePays.value
#On selectionne un aéroport random pour définir un zoom sur le bon pays
try:
    aeroportDsPays = filtreDictAeroports('Country',pays)
    lat = dict_aeroports[aeroportDsPays[0]]['Lat']
    lon = dict_aeroports[aeroportDsPays[0]]['Lon']
except:
    lat = 0
    lon = 0


m = f.Map(
    location = [lat, lon],
    zoom_start = 5,
    min_zoom  = 2,
    min_lot=-200,
    max_lot=200,
    min_lat=-90,
    max_lat=90,
    max_bounds=True,
)

f.TileLayer('cartodbdark_matter').add_to(m)
map_connectionPays(pays,m)
f.LayerControl().add_to(m)

m

### Pays les plus reliés

In [16]:
mr.Md(f"#### Depuis {S_namePays.value}")

#### Depuis Germany

In [17]:
def ChoroplethNbrLiaisons(pays,m):
    '''
    Fonction : Affiche une carte choroplèthe sur le nombre de liaisons entre les pays
    '''

    #Chargement de la carte
    world = "custom.geo.json"
    
    #Mise en forme de la df
    df_tmp = pd.merge(pd.DataFrame.from_dict(dict_aeroports,orient='index')['Country'],
            df_vols[df_vols["Depart"].isin(filtreDictAeroports('Country',pays))],
            left_index=True,
            right_on="Arrivee").groupby('Country').size()

    #errores = 'ignore' car certains pays n'ont pas de vol interne. (Cf Singapour)
    df_tmp.drop(pays, inplace=True, errors='ignore')

    f.Choropleth(
        geo_data=world,
        name="choropleth",
        data=df_tmp,
        key_on="feature.properties.name",
        fill_color="YlOrRd",
        fill_opacity=0.7,
        line_opacity=.1,
        legend_name="Nombre de liaisons",
        nan_fill_opacity = 0.1,
        nan_fill_color='white',
        Highlight= True
    ).add_to(m)


pays = S_namePays.value
m = f.Map(location=[0, 0], 
            zoom_start=2,
            min_zoom  = 2,
            min_lot=-200,
            max_lot=200,
            min_lat=-90,
            max_lat=90,
            max_bounds=True,)
ChoroplethNbrLiaisons(pays,m)
m.get_root().html.add_child(f.Element("<h3 align='center' style='font-size:15px' ><b>Liaisons depuis {}</b></h3>".format(pays)))
f.LayerControl().add_to(m)
m